In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from skimage import io, transform
from torchvision import transforms

import pandas as pd 
import numpy as np
import cv2

import glob

c:\Users\nikla\anaconda3\envs\NLP\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'skimage'

In [200]:
path = "archive/"
data_path = path + "socal2.csv"
pics_path = path + "socal2/socal_pics/"
df = pd.read_csv(data_path)
prices = df["price"]

In [201]:
df.head()

,image_id,street,citi,n_citi,bed,bath,sqft,price
0,0,1317 Van Buren Avenue,"Salton City, CA",317,3,2.0,1560,201900
1,1,124 C Street W,"Brawley, CA",48,3,2.0,713,228500
2,2,2304 Clark Road,"Imperial, CA",152,3,1.0,800,273950
3,3,755 Brawley Avenue,"Brawley, CA",48,3,1.0,1082,350000
4,4,2207 R Carrillo Court,"Calexico, CA",55,4,3.0,2547,385100


In [202]:
X_house_images=np.zeros((15474,64,64,3),dtype='uint32')
cnt=0
for i in range(15474):

    sample=cv2.imread(pics_path+'/'+str(i)+'.jpg')
    imgs=cv2.resize(sample,(64,64))
  
    X_house_images[cnt]=imgs
    cnt+=1

print("No. of images: ",cnt)

No. of images:  15474


In [203]:
from sklearn.model_selection import train_test_split
split = train_test_split(df, X_house_images, test_size=0.25, random_state=42)
(Xatt_train,Xatt_test,Ximage_train,Ximage_test) = split

y_train , y_test = Xatt_train['price'].values , Xatt_test['price'].values

X1_train=Xatt_train[['n_citi','bed','bath','sqft']].values
X2_train=Ximage_train
X1_test=Xatt_test[['n_citi','bed','bath','sqft']].values
X2_test=Ximage_test

In [204]:
X1_train

array([[4.110e+02, 3.000e+00, 2.000e+00, 2.502e+03],
       [3.070e+02, 4.000e+00, 3.000e+00, 1.895e+03],
       [7.800e+01, 4.000e+00, 3.000e+00, 1.573e+03],
       ...,
       [8.200e+01, 4.000e+00, 3.000e+00, 1.550e+03],
       [1.930e+02, 3.000e+00, 2.000e+00, 1.534e+03],
       [2.650e+02, 3.000e+00, 2.100e+00, 2.580e+03]])

In [205]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, x, img, y):
        self.x = torch.tensor(x).float()
        self.img = img
        self.y = torch.tensor(y).float()

    def __len__(self):
        return len(self.x)

    
    def __getitem__(self, idx):
        x = self.x[idx]
        y = self.y[idx]
        img = self.img[idx]
        img = transforms.functional.to_tensor(img.astype(np.uint8).reshape((64, 64, 3)))
        return {'x': x, 'y': y, 'img': img}
    
BATCH_SIZE = 256    
train_dataset = MyDataset(X1_train,X2_train, y_train)
dataLoader_train = torch.utils.data.DataLoader(train_dataset,
                                               batch_size=BATCH_SIZE,
                                               shuffle=True)

test_dataset = MyDataset(X1_test,X2_test, y_test)
dataLoader_test = torch.utils.data.DataLoader(test_dataset,
                                              batch_size=BATCH_SIZE,
                                              shuffle=True)

In [206]:
dataLoader_train


In [22]:
del df['street']
del df['citi']
#del df['n_citi']
df.head()

,image_id,n_citi,bed,bath,sqft,price
0,0,317,3,2.0,1560,201900
1,1,48,3,2.0,713,228500
2,2,152,3,1.0,800,273950
3,3,48,3,1.0,1082,350000
4,4,55,4,3.0,2547,385100


In [70]:
class Model(torch.nn.Module):
    
    def __init__(self, input_shape):
        super().__init__()
        
        self.conv = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=3, out_channels=32, kernel_size=(3, 3)),
            torch.nn.ReLU(),
            torch.nn.BatchNorm2d(32),
            torch.nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(5, 5)),
            torch.nn.ReLU(),
            torch.nn.BatchNorm2d(64),
        )
        
        self.flatten = torch.nn.Sequential(torch.nn.AdaptiveMaxPool2d(1), torch.nn.Flatten())
        
        self.fc = torch.nn.Sequential(
            torch.nn.Linear(input_shape, 256),
            torch.nn.ReLU(),
            torch.nn.Linear(256, 128),
            torch.nn.ReLU(),
        )
        
        self.final_fc = torch.nn.Sequential(
            torch.nn.Linear(128+64, 512),
            torch.nn.ReLU(),
            torch.nn.Linear(512, 1)
        )
        
    def forward(self, x, img):
        img = self.conv(img)
        img = self.flatten(img) 
        x = self.fc(x)
        combined = torch.cat((img, x), dim=1)
        #combined = torch.cat((img.view(img.size(0), -1), x.view(x.size(0), -1)), dim=1)
        #print(x.size(), img.size())
        price = self.final_fc(combined)
        return price
    
model = Model(4)
print(model)
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=5e-4)

Model(
  (conv): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
    (4): ReLU()
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (flatten): Sequential(
    (0): AdaptiveMaxPool2d(output_size=1)
    (1): Flatten(start_dim=1, end_dim=-1)
  )
  (fc): Sequential(
    (0): Linear(in_features=4, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=128, bias=True)
    (3): ReLU()
  )
  (final_fc): Sequential(
    (0): Linear(in_features=192, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=1, bias=True)
  )
)


In [71]:
from tqdm import tqdm
n_epochs = 10
print('started!')
for epoch in tqdm(range(n_epochs)):
    train_batch_loss = 0
    model.train()
    for step, batch in enumerate(dataLoader_train):
        x = batch["x"]
        img = batch["img"]
        y = batch["y"]


        optimizer.zero_grad()
        outputs = model(x = x, img = img)
        loss = criterion(outputs[:,0], y)
        loss.backward()
        optimizer.step()
        train_batch_loss += loss.item()
        optimizer.step()
        optimizer.zero_grad()

    test_batch_loss = 0
    model.eval()

    with torch.no_grad():
         for step, batch in enumerate(dataLoader_test):
            x = batch["x"]
            img = batch["img"]
            y = batch["y"]
            outputs = model(x = x, img = img)
            loss = criterion(outputs[:,0], y)
            test_batch_loss += loss.item()     

    print('epoch {}/{} finished with train loss: {} and test loss: {}'.format(epoch+1, n_epochs, train_batch_loss / len(dataLoader_train), test_batch_loss / len(dataLoader_test)))
    
torch.save(model.state_dict(), './model_two_input.pt')

started!


 10%|█         | 1/10 [02:39<23:58, 159.80s/it]

epoch 1/10 finished with train loss: 525078970724.1739 and test loss: 207061161472.0


 20%|██        | 2/10 [05:23<21:36, 162.05s/it]

epoch 2/10 finished with train loss: 110927604869.56522 and test loss: 107115519744.0


...

epoch 8/10 finished with train loss: 95159813787.82608 and test loss: 99372399104.0


 90%|█████████ | 9/10 [24:17<02:42, 162.56s/it]

epoch 9/10 finished with train loss: 93755081505.39131 and test loss: 99780259328.0


100%|██████████| 10/10 [26:53<00:00, 161.37s/it]

epoch 10/10 finished with train loss: 93853830544.69565 and test loss: 99534354944.0


In [72]:
import math

In [73]:
def reg_report(true, pred, name='Test'):
    print("\n{} Results :\n".format(name))
    print("RSS :",sum((pred-true)**2))
    print("RSE :",math.sqrt(sum((pred-true)**2)*(1/(len(pred)-2))))
    print("TSS :",sum((true-true.mean())**2))
    print("R Squared :",1-(sum((pred-true)**2)/sum((true-true.mean())**2)))
    print("MSE :",((pred-true)**2).mean())
    print('MAE :',(abs(pred-true)).mean())
    print('Accuracy with 10% :', ((pred<=true*1.1) & (true*0.9<=pred)).mean())
    

def eval_report(y_train, pred_train,y_test, pred_test):
    reg_report(y_train, pred_train, name='Train')
    reg_report(y_test, pred_test, name='Test')
    
def res(dataLoader, name = 'Test'):  
    trues = []
    preds = []
    model.eval()
    with torch.no_grad():
        for step, batch in enumerate(dataLoader):
            x = batch["x"]
            img = batch["img"]
            y = batch["y"]

            outputs = model(x = x, img = img)

            trues = trues + y.tolist()
            preds = preds + outputs[:,0].tolist()


    reg_report(true =  np.array(trues), pred = np.array(preds), name=name)

res(dataLoader_test, name = 'Test')
res(dataLoader_train, name = 'Train')


Test Results :

RSS : 380696832934998.3
RSE : 313763.5903382174
TSS : 560633939775361.56
R Squared : 0.3209529321618695
MSE : 98396700164.12643
MAE : 229841.06415578962
Accuracy with 10% : 0.18350995089170327

Train Results :

RSS : 1075644050759336.2
RSE : 304473.250464067
TSS : 1638210861992667.8
R Squared : 0.3434031749423533
MSE : 92687983693.1802
MAE : 224996.14421450882
Accuracy with 10% : 0.1912968548039638


In [118]:
class Netz(nn.Module):
    def __init__(self, input_shape):
        super(Netz, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, kernel_size=5)
        self.conv2 = nn.Conv2d(6, 9, kernel_size=5)
        #self.flatten=nn.Sequential(nn.AdaptiveMaxPool2d(1), nn.Flatten())
        self.fc1 = nn.Linear(input_shape, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(137, 1)


    def forward(self, x, img):
        img = self.conv1(img)
        img = F.max_pool2d(img, 3)
        img = F.relu(img)

        img = self.conv2(img)
        img = F.max_pool2d(img, 3)
        img = F.relu(img)
        #
        # img=self.flatten(img)

        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        #rint(x.shape,img.shape)
        #combined = torch.cat((img.view(img.size(0), -1), x.view(x.size(0), -1)), dim=1)
        combined = torch.cat((x, img), 1)
        price = self.fc3(combined)
        return price
        #pass

model_2 = Netz(4)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)
print(model)

Model(
  (conv): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
    (4): ReLU()
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (flatten): Sequential(
    (0): AdaptiveMaxPool2d(output_size=1)
    (1): Flatten(start_dim=1, end_dim=-1)
  )
  (fc): Sequential(
    (0): Linear(in_features=4, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=128, bias=True)
    (3): ReLU()
  )
  (final_fc): Sequential(
    (0): Linear(in_features=192, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=1, bias=True)
  )
)


In [119]:
from tqdm import tqdm
n_epochs = 10
print('started!')
for epoch in tqdm(range(n_epochs)):
    train_batch_loss = 0
    model_2.train()
    for step, batch in enumerate(dataLoader_train):
        x = batch["x"]
        img = batch["img"]
        y = batch["y"]


        optimizer.zero_grad()
        outputs = model_2(x = x, img = img)
        loss = criterion(outputs[:,0], y)
        loss.backward()
        optimizer.step()
        train_batch_loss += loss.item()
        optimizer.step()
        optimizer.zero_grad()

    test_batch_loss = 0
    model_2.eval()

    with torch.no_grad():
         for step, batch in enumerate(dataLoader_test):
            x = batch["x"]
            img = batch["img"]
            y = batch["y"]
            outputs = model_2(x = x, img = img)
            loss = criterion(outputs[:,0], y)
            test_batch_loss += loss.item()     

    print('epoch {}/{} finished with train loss: {} and test loss: {}'.format(epoch+1, n_epochs, train_batch_loss / len(dataLoader_train), test_batch_loss / len(dataLoader_test)))
    
torch.save(model_2.state_dict(), './mmodel_2odel_two_input.pt')

started!


  0%|          | 0/10 [00:00<?, ?it/s]


RuntimeError: Tensors must have same number of dimensions: got 2 and 4

In [ ]:
import math

In [115]:
def reg_report(true, pred, name='Test'):
    print("\n{} Results :\n".format(name))
    print("RSS :",sum((pred-true)**2))
    print("RSE :",math.sqrt(sum((pred-true)**2)*(1/(len(pred)-2))))
    print("TSS :",sum((true-true.mean())**2))
    print("R Squared :",1-(sum((pred-true)**2)/sum((true-true.mean())**2)))
    print("MSE :",((pred-true)**2).mean())
    print('MAE :',(abs(pred-true)).mean())
    print('Accuracy with 10% :', ((pred<=true*1.1) & (true*0.9<=pred)).mean())
    

def eval_report(y_train, pred_train,y_test, pred_test):
    reg_report(y_train, pred_train, name='Train')
    reg_report(y_test, pred_test, name='Test')
    
def res(dataLoader, name = 'Test'):  
    trues = []
    preds = []
    model_2.eval()
    with torch.no_grad():
        for step, batch in enumerate(dataLoader):
            x = batch["x"]
            img = batch["img"]
            y = batch["y"]

            outputs = model_2(x = x, img = img)

            trues = trues + y.tolist()
            preds = preds + outputs[:,0].tolist()


    reg_report(true =  np.array(trues), pred = np.array(preds), name=name)

res(dataLoader_test, name = 'Test')
res(dataLoader_train, name = 'Train')


Test Results :

RSS : 2488957288173446.0
RSE : 802272.0013507858
TSS : 560633939775360.8
R Squared : -3.439540868986171
MSE : 643307647498.9537
MAE : 705963.4366304198
Accuracy with 10% : 0.0

Train Results :

RSS : 7362237494844666.0
RSE : 796562.3317061164
TSS : 1638210861992678.0
R Squared : -3.4940719571895817
MSE : 634402196884.5054
MAE : 702295.735096819
Accuracy with 10% : 0.0


In [130]:
class TwoInputsNet(nn.Module):
    def __init__(self):
        super(TwoInputsNet, self).__init__()
        self.conv = nn.Conv2d(3,8,kernel_size=3) 
        self.conv1 = nn.Conv2d(8,8,kernel_size=3)
        self.conv2 = nn.Conv2d(8,8,kernel_size=3) 
        self.fc1 = nn.Linear(3,3)
        self.fc2 = nn.Linear(26915,1024)
        self.fc3 = nn.Linear(1024,32) 
        self.fc4 = nn.Linear(32,1) 

    def forward(self, input1, input2):
        c = self.conv(input1)
        c = self.conv1(c)
        c = F.relu(c)
        c = self.conv2(c)
        c = F.relu(c)
        f = self.fc1(input2)
        # now we can reshape `c` and `f` to 2D and concat them
        #combined = torch.cat((c.view(c.size(0), -1), f.view(f.size(0), -1)), dim=1)
        combined = torch.cat((img, x), dim=1)
        out = self.fc2(combined)
        out = F.relu(out)
        out = self.fc3(out)
        out = F.relu(out)
        out = self.fc4(out)

        
        return out

In [131]:
network = TwoInputsNet()
print(network)

TwoInputsNet(
  (conv): Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1))
  (conv1): Conv2d(8, 8, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(8, 8, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=3, out_features=3, bias=True)
  (fc2): Linear(in_features=26915, out_features=1024, bias=True)
  (fc3): Linear(in_features=1024, out_features=32, bias=True)
  (fc4): Linear(in_features=32, out_features=1, bias=True)
)


In [132]:
learning_rate = 0.01
momentum = 0.9
n_epochs = 10

optimizer = optim.SGD(network.parameters(), lr=learning_rate,momentum=momentum)

loss_func = torch.nn.MSELoss()

In [133]:
def train(dataloader,epoch):
    batch_idx = 0
    for items in dataloader:
        image = torch.FloatTensor(batch['img'])
        features = torch.FloatTensor(items['x'])
        price = torch.FloatTensor(items['y'])

        if torch.cuda.is_available():
            image = image.cuda()
            features = features.cuda()
            price = price.cuda()
            
        output = network(image,features)
        output = output.reshape(4)
        loss = loss_func(output, price)
        optimizer.zero_grad()  
        loss.backward()
        optimizer.step()
        
        if batch_idx % 4 == 0: #every 25 * batchsize sample we print results
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch+1, batch_idx * image.shape[0], len(dataloader.dataset),
            100. * batch_idx / len(dataloader), loss.item()))
            
        batch_idx = batch_idx + 1

In [134]:
for epoch in range(n_epochs):
    # train 
    train(train_dataset,epoch)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x4 and 3x3)

In [224]:
df.head()

,image_id,n_citi,bed,bath,sqft,price
0,0,0.234300,3,2.0,0.926382,0.996177
1,1,0.884058,3,2.0,0.975096,0.981440
2,2,0.632850,3,1.0,0.970093,0.956260
3,3,0.884058,3,1.0,0.953874,0.914127
4,4,0.867150,4,3.0,0.869615,0.894681


In [225]:
df = df.sample(256)
df.reset_index(inplace = True )
del df['index']
df.head()

,image_id,n_citi,bed,bath,sqft,price
0,10305,0.811594,4,2.1,0.895439,0.693629
1,7918,0.328502,2,2.0,0.938805,0.582271
2,8316,0.236715,3,2.0,0.949272,0.994460
3,6262,0.016908,4,3.0,0.855984,0.858726
4,1938,0.321256,3,2.0,0.952091,0.803380


In [226]:
del df['street']
del df['citi']
for cols in ['sqft','price','n_citi']:
    df[cols] = (df[cols].max() - df[cols])/(df[cols].max() - df[cols].min())
    
df.head()

KeyError: 'street'

In [ ]:
class SocalDataset(Dataset):
    
    def __init__(self, dataframe, root_dir, transform=None):
        self.features = dataframe
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        try:
            img_name = '{}{}.jpg'.format(str(self.root_dir), str(self.features.loc[idx,'image_id']))
            image = io.imread(img_name)
            image = image / 255.0
        
            if len(image.shape) == 3:
                house_features = self.features.iloc[idx, 1:]
                house_features = np.array([house_features]).reshape(5)
                sample = {'image': image, 'house_features': house_features}

                if self.transform:
                    sample['image'] = transform.resize(sample['image'],(64,64)).reshape(3,64,64)
                    
                sample['image'] = torch.from_numpy(sample['image']).float()
                sample['house_features'] = torch.from_numpy(sample['house_features']).float()

                return sample
        except:
            pass

In [227]:
house_dataset = SocalDataset(dataframe=df,
                            root_dir=pics_path,
                            transform = True)

In [228]:
for i in range(10):
    sample = house_dataset[i]
    print(i, sample['image'].shape, sample['house_features'].shape)

0 torch.Size([3, 64, 64]) torch.Size([5])
1 torch.Size([3, 64, 64]) torch.Size([5])
2 torch.Size([3, 64, 64]) torch.Size([5])
3 torch.Size([3, 64, 64]) torch.Size([5])
4 torch.Size([3, 64, 64]) torch.Size([5])
5 torch.Size([3, 64, 64]) torch.Size([5])
6 torch.Size([3, 64, 64]) torch.Size([5])
7 torch.Size([3, 64, 64]) torch.Size([5])
8 torch.Size([3, 64, 64]) torch.Size([5])
9 torch.Size([3, 64, 64]) torch.Size([5])


In [229]:
test_size = 0.3

test_amount = int(house_dataset.__len__() * test_size)

train_set, test_set = torch.utils.data.random_split(house_dataset,[
            (house_dataset.__len__() - test_amount ), test_amount ])

In [230]:
train_dataloader = torch.utils.data.DataLoader(
            train_set,
            batch_size=5,
            shuffle=True,
)

test_dataloader = torch.utils.data.DataLoader(
            test_set,
            batch_size=5,
            shuffle=True,
)

In [231]:
it = iter(train_dataloader)
items = next(it)
print(type(items))

<class 'dict'>


In [232]:
print(items['image'].shape)
print(items['house_features'].shape)
print(items['house_features'])

torch.Size([5, 3, 64, 64])
torch.Size([5, 5])
tensor([[0.8019, 3.0000, 3.0000, 0.9127, 0.9668],
        [0.8744, 2.0000, 1.0000, 0.9431, 0.6931],
        [0.3599, 2.0000, 1.0000, 0.9435, 0.7618],
        [0.5338, 3.0000, 2.0000, 0.9416, 0.9778],
        [0.4372, 3.0000, 2.1000, 0.9174, 0.8643]])


In [233]:
class TwoInputsNet(nn.Module):
    def __init__(self):
        super(TwoInputsNet, self).__init__()
        self.conv = nn.Conv2d(3,6,kernel_size=3) 
        self.conv1 = nn.Conv2d(6,9,kernel_size=3)
        self.conv2 = nn.Conv2d(9,12,kernel_size=3) 
        self.fc1 = nn.Linear(4,3)
        self.fc2 = nn.Linear(40371,1024)
        self.fc3 = nn.Linear(1024,32) 
        self.fc4 = nn.Linear(32,1) 

    def forward(self, input1, input2):
        c = self.conv(input1)
        c = self.conv1(c)
        c = F.relu(c)
        c = self.conv2(c)
        c = F.relu(c)
        f = self.fc1(input2)
        
        # now we can reshape `c` and `f` to 2D and concat them
        combined = torch.cat((c.view(c.size(0), -1),
                          f.view(f.size(0), -1)), dim=1)
        out = self.fc2(combined)
        out = F.relu(out)
        out = self.fc3(out)
        out = F.relu(out)
        out = self.fc4(out)

        
        return out

In [234]:
network = TwoInputsNet()
print(network)

TwoInputsNet(
  (conv): Conv2d(3, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv1): Conv2d(6, 9, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(9, 12, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=4, out_features=3, bias=True)
  (fc2): Linear(in_features=40371, out_features=1024, bias=True)
  (fc3): Linear(in_features=1024, out_features=32, bias=True)
  (fc4): Linear(in_features=32, out_features=1, bias=True)
)


In [235]:
learning_rate = 0.01
momentum = 0.9
n_epochs = 5

optimizer = optim.SGD(network.parameters(), lr=learning_rate,momentum=momentum)

loss_func = torch.nn.MSELoss()

In [236]:
train_dataloader = torch.utils.data.DataLoader(
            train_set,
            batch_size=5,
            shuffle=True,
)

test_dataloader = torch.utils.data.DataLoader(
            test_set,
            batch_size=5,
            shuffle=True,
)

In [239]:
def train(dataloader,epoch):
    batch_idx = 0
    for items in dataloader:
        image = torch.FloatTensor(items['image'])
        features = torch.FloatTensor(items['house_features'][:,:4])
        price = torch.FloatTensor(items['house_features'][:,4])

        if torch.cuda.is_available():
            image = image.cuda()
            features = features.cuda()
            price = price.cuda()
            
        output = network(image,features)
        output = output.reshape(5)
        loss = loss_func(output, price)
        optimizer.zero_grad()  
        loss.backward()
        optimizer.step()
        
        if batch_idx % 4 == 0: #every 25 * batchsize sample we print results
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch+1, batch_idx * image.shape[0], len(dataloader.dataset),
            100. * batch_idx / len(dataloader), loss.item()))
            
        batch_idx = batch_idx + 1

In [241]:
for epoch in range(n_epochs):
    # train 
    train(train_dataloader,epoch)

Train Epoch: 1 [0/180 (0%)]	Loss: 0.452995
Train Epoch: 1 [20/180 (11%)]	Loss: 0.126921
Train Epoch: 1 [40/180 (22%)]	Loss: 0.093897
Train Epoch: 1 [60/180 (33%)]	Loss: 0.059054
Train Epoch: 1 [80/180 (44%)]	Loss: 0.018912
Train Epoch: 1 [100/180 (56%)]	Loss: 0.026393
Train Epoch: 1 [120/180 (67%)]	Loss: 0.073876
Train Epoch: 1 [140/180 (78%)]	Loss: 0.019582
Train Epoch: 1 [160/180 (89%)]	Loss: 0.071252
Train Epoch: 2 [0/180 (0%)]	Loss: 0.009247
Train Epoch: 2 [20/180 (11%)]	Loss: 0.005556
Train Epoch: 2 [40/180 (22%)]	Loss: 0.057850
Train Epoch: 2 [60/180 (33%)]	Loss: 0.207975
Train Epoch: 2 [80/180 (44%)]	Loss: 0.030397
Train Epoch: 2 [100/180 (56%)]	Loss: 0.086041
Train Epoch: 2 [120/180 (67%)]	Loss: 0.005431
Train Epoch: 2 [140/180 (78%)]	Loss: 0.035753
Train Epoch: 2 [160/180 (89%)]	Loss: 0.133349
Train Epoch: 3 [0/180 (0%)]	Loss: 0.015860
Train Epoch: 3 [20/180 (11%)]	Loss: 0.035526
Train Epoch: 3 [40/180 (22%)]	Loss: 0.020284
Train Epoch: 3 [60/180 (33%)]	Loss: 0.011326
Train Ep

In [242]:
import math
def reg_report(true, pred, name='Test'):
    print("\n{} Results :\n".format(name))
    print("RSS :",sum((pred-true)**2))
    print("RSE :",math.sqrt(sum((pred-true)**2)*(1/(len(pred)-2))))
    print("TSS :",sum((true-true.mean())**2))
    print("R Squared :",1-(sum((pred-true)**2)/sum((true-true.mean())**2)))
    print("MSE :",((pred-true)**2).mean())
    print('MAE :',(abs(pred-true)).mean())
    print('Accuracy with 10% :', ((pred<=true*1.1) & (true*0.9<=pred)).mean())
    

def eval_report(y_train, pred_train,y_test, pred_test):
    reg_report(y_train, pred_train, name='Train')
    reg_report(y_test, pred_test, name='Test')
    
def res(dataLoader, name = 'Test'):  
    trues = []
    preds = []
    network.eval()
    with torch.no_grad():
        for items in dataLoader:
            image = torch.FloatTensor(items['image'])
            features = torch.FloatTensor(items['house_features'][:,:4])
            price = torch.FloatTensor(items['house_features'][:,4])

            outputs = network(input2 = features, input1 = image)

            trues = trues + price.tolist()
            preds = preds + outputs[:,0].tolist()


    reg_report(true =  np.array(trues), pred = np.array(preds), name=name)

res(train_dataloader, name = 'Test')
res(test_dataloader, name = 'Train')


Test Results :

RSS : 6.53765385536986
RSE : 0.19164652996049286
TSS : 6.631032657354906
R Squared : 0.014082090499354383
MSE : 0.03632029919649922
MAE : 0.13936191845441656
Accuracy with 10% : 0.35555555555555557

Train Results :

RSS : 3.6185278102910923
RSE : 0.22113123809944435
TSS : 3.641416601597961
R Squared : 0.006285683241193718
MSE : 0.047612208030145944
MAE : 0.16736403372334807
Accuracy with 10% : 0.27631578947368424


[W NNPACK.cpp:53] Could not initialize NNPACK! Reason: Unsupported hardware.
